In [30]:
import json
import os
import time
from requests import get
from bs4 import BeautifulSoup
from dataclasses import dataclass
from typing import List

# Récupérer les liens des matchs 

In [31]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
print(current_dir,parent_dir)

d:\Cours_M2\webscraping\ml-webscrap-tennis\scraping d:\Cours_M2\webscraping\ml-webscrap-tennis


In [32]:
path_detail_joueurs = os.path.join(parent_dir, "donnees", "detail_joueurs.json")
with open(path_detail_joueurs, "r") as fichier:
    detail_joueurs = json.load(fichier)

In [33]:
liens_match = set()
for key, value in detail_joueurs.items():
    joueur = detail_joueurs[key]
    matchs = joueur['matchs']
    
    for i in range(len(matchs)):
        lien_match = matchs[i]['lien_detail_match']
        liens_match.add(lien_match)

In [34]:
liens_match

{'https://www.tennisendirect.net/atp/match/alejandro-davidovich-fokina-VS-daniil-medvedev/national-bank-open-montreal-2024/',
 'https://www.tennisendirect.net/atp/match/alejandro-tabilo-VS-novak-djokovic/internazionali-bnl-ditalia-rome-2024/',
 'https://www.tennisendirect.net/atp/match/aleksandar-vukic-VS-casper-ruud/shanghai-rolex-masters-shanghai-2024/',
 'https://www.tennisendirect.net/atp/match/alex-de-minaur-VS-alex-michelsen/french-open-paris-2024/',
 'https://www.tennisendirect.net/atp/match/alex-de-minaur-VS-alexander-bublik/bnp-paribas-open-indian-wells-2024/',
 'https://www.tennisendirect.net/atp/match/alex-de-minaur-VS-alexei-popyrin/monte-carlo-rolex-masters-monte-carlo-2024/',
 'https://www.tennisendirect.net/atp/match/alex-de-minaur-VS-arthur-fils/wimbledon-london-2024/',
 'https://www.tennisendirect.net/atp/match/alex-de-minaur-VS-daniel-evans/us-open-new-york-2024/',
 'https://www.tennisendirect.net/atp/match/alex-de-minaur-VS-daniil-medvedev/french-open-paris-2024/',
 

# Test de scrap sur un match : match_1

In [35]:
id_matchs = {f"match_{i+1}": lien for i, lien in enumerate(liens_match)}
id_matchs

{'match_1': 'https://www.tennisendirect.net/atp/match/carlos-alcaraz-garfia-VS-hady-habib/olympics-paris-2024/',
 'match_2': 'https://www.tennisendirect.net/atp/match/jakub-mensik-VS-grigor-dimitrov/mutua-madrid-open-madrid-2024/',
 'match_3': 'https://www.tennisendirect.net/atp/match/luca-nardi-VS-novak-djokovic/bnp-paribas-open-indian-wells-2024/',
 'match_4': 'https://www.tennisendirect.net/atp/match/alexander-zverev-VS-ben-shelton/western-southern-open-cincinnati-2024/',
 'match_5': 'https://www.tennisendirect.net/atp/match/alexander-zverev-VS-casper-ruud/french-open-paris-2024/',
 'match_6': 'https://www.tennisendirect.net/atp/match/casper-ruud-VS-taro-taro/olympics-paris-2024/',
 'match_7': 'https://www.tennisendirect.net/atp/match/jannik-sinner-VS-christopher-oconnell/us-open-new-york-2024/',
 'match_8': 'https://www.tennisendirect.net/atp/match/carlos-alcaraz-garfia-VS-juncheng-shang/shanghai-rolex-masters-shanghai-2024/',
 'match_9': 'https://www.tennisendirect.net/atp/match/c

In [55]:
premiers_liens_avec_id = {key: id_matchs[key] for key in list(id_matchs.keys())[:10]}
premiers_liens_avec_id

{'match_1': 'https://www.tennisendirect.net/atp/match/carlos-alcaraz-garfia-VS-hady-habib/olympics-paris-2024/',
 'match_2': 'https://www.tennisendirect.net/atp/match/jakub-mensik-VS-grigor-dimitrov/mutua-madrid-open-madrid-2024/',
 'match_3': 'https://www.tennisendirect.net/atp/match/luca-nardi-VS-novak-djokovic/bnp-paribas-open-indian-wells-2024/',
 'match_4': 'https://www.tennisendirect.net/atp/match/alexander-zverev-VS-ben-shelton/western-southern-open-cincinnati-2024/',
 'match_5': 'https://www.tennisendirect.net/atp/match/alexander-zverev-VS-casper-ruud/french-open-paris-2024/',
 'match_6': 'https://www.tennisendirect.net/atp/match/casper-ruud-VS-taro-taro/olympics-paris-2024/',
 'match_7': 'https://www.tennisendirect.net/atp/match/jannik-sinner-VS-christopher-oconnell/us-open-new-york-2024/',
 'match_8': 'https://www.tennisendirect.net/atp/match/carlos-alcaraz-garfia-VS-juncheng-shang/shanghai-rolex-masters-shanghai-2024/',
 'match_9': 'https://www.tennisendirect.net/atp/match/c

In [36]:
matchs = list(liens_match)
match_1 = matchs[0]
match_1

'https://www.tennisendirect.net/atp/match/carlos-alcaraz-garfia-VS-hady-habib/olympics-paris-2024/'

## Récupération des stats des 2 joueurs sur le match 

In [37]:
@dataclass
class StatsMatch:
    nom_joueur: str
    premier_service: str
    pnts_gagnes_ps: str
    pnts_gagnes_ss: str
    balles_break_gagnees: str
    retours_gagnes: str
    total_points_gagnes: str
    double_fautes: str
    aces: str

In [38]:
def extraire_colonnes(table: BeautifulSoup) -> List[str]:
    """
    Extrait les données des colonnes sous forme de liste de texte brut.
    """
    return [td.text.strip() for td in table.find_all("td")]

In [39]:
def organiser_stats_par_joueur(table_data: List[str]) -> tuple:
    """
    Réorganise les données extraites en deux listes, une pour chaque joueur.
    """
    joueur_A, joueur_B = [], []
    for i, data in enumerate(table_data):
        if i % 3 == 1:  # Colonne du joueur A
            joueur_A.append(data)
        elif i % 3 == 2:  # Colonne du joueur B
            joueur_B.append(data)
    return joueur_A, joueur_B

In [40]:
def creer_stats_joueur(joueur_data: List[str]) -> StatsMatch:
    """
    Crée une instance de StatsMatch pour un joueur à partir des données extraites.
    """
    return StatsMatch(
        nom_joueur=joueur_data[0],
        premier_service=joueur_data[1],
        pnts_gagnes_ps=joueur_data[2],
        pnts_gagnes_ss=joueur_data[3],
        balles_break_gagnees=joueur_data[4],
        retours_gagnes=joueur_data[5],
        total_points_gagnes=joueur_data[6],
        double_fautes=joueur_data[7],
        aces=joueur_data[8],
    )

In [41]:
reponse = get(match_1)
assert reponse.status_code == 200, f"Erreur HTTP {reponse.status_code}: {match_1}"

In [42]:
detail_match = BeautifulSoup(reponse.text, features="lxml")
statistiques = detail_match.find_all("table", attrs={"class": "table_stats_match"})[0]

In [43]:
table_data = extraire_colonnes(statistiques)
assert len(table_data) % 3 == 0
joueur_A_data, joueur_B_data = organiser_stats_par_joueur(table_data)

In [56]:
table_data

['',
 'Carlos Alcaraz',
 'Hady Habib',
 'premier service en pourcentage',
 '33/47 (70%)',
 '40/63 (63%)',
 'Points gagnés sur 1er service',
 '26/33 (79%)',
 '21/40 (53%)',
 'Points gagnés sur 2e service',
 '10/14 (71%)',
 '10/23 (43%)',
 'Balles de break gagnées',
 '4/13 (31%)',
 '0/4 (0%)',
 'Points gagnés sur retour',
 '32/63 (51%)',
 '11/47 (23%)',
 'Total de points gagnés',
 '68/110 (62%)',
 '42/110 (38%)',
 'Double fautes',
 '2',
 '3',
 'Aces',
 '5',
 '3']

In [44]:
stats_joueur_A: StatsMatch = creer_stats_joueur(joueur_A_data)
stats_joueur_B: StatsMatch = creer_stats_joueur(joueur_B_data)

print(stats_joueur_A)
print(stats_joueur_B)

StatsMatch(nom_joueur='Carlos Alcaraz', premier_service='33/47 (70%)', pnts_gagnes_ps='26/33 (79%)', pnts_gagnes_ss='10/14 (71%)', balles_break_gagnees='4/13 (31%)', retours_gagnes='32/63 (51%)', total_points_gagnes='68/110 (62%)', double_fautes='2', aces='5')
StatsMatch(nom_joueur='Hady Habib', premier_service='40/63 (63%)', pnts_gagnes_ps='21/40 (53%)', pnts_gagnes_ss='10/23 (43%)', balles_break_gagnees='0/4 (0%)', retours_gagnes='11/47 (23%)', total_points_gagnes='42/110 (38%)', double_fautes='3', aces='3')


## Stocker les stats avec le match 

En mode : 
    id match :{
        stats_joueur_A
        stats_joueur_B
    }

In [45]:
stats_joueur_A.__dict__

{'nom_joueur': 'Carlos Alcaraz',
 'premier_service': '33/47 (70%)',
 'pnts_gagnes_ps': '26/33 (79%)',
 'pnts_gagnes_ss': '10/14 (71%)',
 'balles_break_gagnees': '4/13 (31%)',
 'retours_gagnes': '32/63 (51%)',
 'total_points_gagnes': '68/110 (62%)',
 'double_fautes': '2',
 'aces': '5'}

In [46]:
stats_joueur_B.__dict__

{'nom_joueur': 'Hady Habib',
 'premier_service': '40/63 (63%)',
 'pnts_gagnes_ps': '21/40 (53%)',
 'pnts_gagnes_ss': '10/23 (43%)',
 'balles_break_gagnees': '0/4 (0%)',
 'retours_gagnes': '11/47 (23%)',
 'total_points_gagnes': '42/110 (38%)',
 'double_fautes': '3',
 'aces': '3'}

In [47]:
id_match = match_1
id_match

'https://www.tennisendirect.net/atp/match/carlos-alcaraz-garfia-VS-hady-habib/olympics-paris-2024/'

id_match : {
    stats
}

In [50]:
match_data: dict = {}
match_data["match_1"] = {
    "joueur_gagnant": stats_joueur_A.__dict__,
    "joueur_perdant": stats_joueur_B.__dict__
}

In [51]:
match_data

{'match_1': {'joueur_gagnant': {'nom_joueur': 'Carlos Alcaraz',
   'premier_service': '33/47 (70%)',
   'pnts_gagnes_ps': '26/33 (79%)',
   'pnts_gagnes_ss': '10/14 (71%)',
   'balles_break_gagnees': '4/13 (31%)',
   'retours_gagnes': '32/63 (51%)',
   'total_points_gagnes': '68/110 (62%)',
   'double_fautes': '2',
   'aces': '5'},
  'joueur_perdant': {'nom_joueur': 'Hady Habib',
   'premier_service': '40/63 (63%)',
   'pnts_gagnes_ps': '21/40 (53%)',
   'pnts_gagnes_ss': '10/23 (43%)',
   'balles_break_gagnees': '0/4 (0%)',
   'retours_gagnes': '11/47 (23%)',
   'total_points_gagnes': '42/110 (38%)',
   'double_fautes': '3',
   'aces': '3'}}}

# Test sur une boucle

In [ ]:
current_dir: str = os.getcwd()
file_path: str = os.path.join(current_dir, "donnees", "joueurs.json")
output_file: str = os.path.join(current_dir, "donnees", "stats_matchs.json")

if os.path.exists(output_file):
    with open(output_file, "r", encoding="utf-8") as fichier:
        try:
            joueurs_data = json.load(fichier)
        except json.JSONDecodeError:
            print(f"Le fichier {output_file} est corrompu ou vide. Il sera remplacé.")
            joueurs_data = {}
else:
    joueurs_data = {}

match_data: dict = {}

for id_match, lien_match in id_matchs.items():
    reponse = get(lien_match)
    assert reponse.status_code == 200, f"Erreur HTTP {reponse.status_code}: {lien_match}"
    
    detail_match = BeautifulSoup(reponse.text, features="lxml")
    statistiques = detail_match.find_all("table", attrs={"class": "table_stats_match"})[0]
    
    table_data = extraire_colonnes(statistiques)
    assert len(table_data) % 3 == 0
    joueur_A_data, joueur_B_data = organiser_stats_par_joueur(table_data)
    
    stats_joueur_A: StatsMatch = creer_stats_joueur(joueur_A_data)
    stats_joueur_B: StatsMatch = creer_stats_joueur(joueur_B_data)
    
    match_data[id_match] = {
        "joueur_gagnant": stats_joueur_A.__dict__,
        "joueur_perdant": stats_joueur_B.__dict__
    }
    
    time.sleep(1)


problème dans le tableau des stasts, sur certains il manque une ligne